In [2]:
在处理缺失值之前，我们首先读取数据集文件，分析每一列的属性值并获取其缺失值信息

In [2]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

In [4]:
data = pd.read_csv("..//..//dataset//oakland-crime-statistics-2011-to-2016//records-for-2011.csv")
print(data)

       Agency              Create Time                  Location  Area Id  \
0          OP  2011-01-01T00:00:00.000           ST&SAN PABLO AV      1.0   
1          OP  2011-01-01T00:01:11.000             ST&HANNAH ST       1.0   
2          OP  2011-01-01T00:01:25.000             ST&MARKET ST       1.0   
3          OP  2011-01-01T00:01:35.000              PRENTISS ST       2.0   
4          OP  2011-01-01T00:02:10.000         AV&FOOTHILL BLVD       2.0   
5          OP  2011-01-01T00:02:33.000               OAKLAND AV       1.0   
6          OP  2011-01-01T00:03:03.000                     B ST       3.0   
7          OP  2011-01-01T00:04:25.000         AV&FOOTHILL BLVD       2.0   
8          OP  2011-01-01T00:05:13.000         AV&FOOTHILL BLVD       2.0   
9          OP  2011-01-01T00:05:51.000              AV&TEVIS ST       3.0   
10         OP  2011-01-01T00:06:21.000                 LYNDE ST       2.0   
11         OP  2011-01-01T00:06:26.000         23RD ST&HUGHES AV      2.0   

我们可以注意到原数据集有180016行

#### 1. 删除空值处理

In [19]:
#删除所有属性中有空值的行
data1 = data.dropna()
print(data1)

       Agency              Create Time                  Location  Area Id  \
0          OP  2011-01-01T00:00:00.000           ST&SAN PABLO AV      1.0   
1          OP  2011-01-01T00:01:11.000             ST&HANNAH ST       1.0   
2          OP  2011-01-01T00:01:25.000             ST&MARKET ST       1.0   
3          OP  2011-01-01T00:01:35.000              PRENTISS ST       2.0   
4          OP  2011-01-01T00:02:10.000         AV&FOOTHILL BLVD       2.0   
5          OP  2011-01-01T00:02:33.000               OAKLAND AV       1.0   
6          OP  2011-01-01T00:03:03.000                     B ST       3.0   
7          OP  2011-01-01T00:04:25.000         AV&FOOTHILL BLVD       2.0   
8          OP  2011-01-01T00:05:13.000         AV&FOOTHILL BLVD       2.0   
9          OP  2011-01-01T00:05:51.000              AV&TEVIS ST       3.0   
10         OP  2011-01-01T00:06:21.000                 LYNDE ST       2.0   
11         OP  2011-01-01T00:06:26.000         23RD ST&HUGHES AV      2.0   

删除以后数据集只剩下178771行

#### 2. 用最高频率值进行填充

In [32]:
data2 = copy.deepcopy(data)#拷贝原数据集用于填充
for i in range(10):
    count = data2.iloc[:,i].value_counts()  #依次获取每一列的series
    name = pd.DataFrame(data2.iloc[:,i]).columns.values.tolist()
    sort_ans = count.sort_values(ascending = False) #对各类别的频数值进行排序 并选择最高频率的索引
    fre = sort_ans.index[0]
    fre = str(fre)
    print(name[0]+"的最高频率值为"+fre+'\n')
    
    data2[name[0]] = data2[name[0]].fillna(fre) #进行缺失值的填充
print("------------------------------------------"+'\n'+"填充完毕")

Agency的最高频率值为OP

Create Time的最高频率值为2011-06-02T00:00:00.000

Location的最高频率值为 INTERNATIONAL BLVD 

Area Id的最高频率值为1.0

Beat的最高频率值为04X

Priority的最高频率值为2.0

Incident Type Id的最高频率值为933R

Incident Type Description的最高频率值为ALARM-RINGER

Event Number的最高频率值为LOP110218000226

Closed Time的最高频率值为2011-07-05T12:36:42.000

------------------------------------------
填充完毕


In [33]:
print(data2)  #填充结果

       Agency              Create Time                  Location Area Id Beat  \
0          OP  2011-01-01T00:00:00.000           ST&SAN PABLO AV       1  06X   
1          OP  2011-01-01T00:01:11.000             ST&HANNAH ST        1  07X   
2          OP  2011-01-01T00:01:25.000             ST&MARKET ST        1  10Y   
3          OP  2011-01-01T00:01:35.000              PRENTISS ST        2  21Y   
4          OP  2011-01-01T00:02:10.000         AV&FOOTHILL BLVD        2  20X   
5          OP  2011-01-01T00:02:33.000               OAKLAND AV        1  08X   
6          OP  2011-01-01T00:03:03.000                     B ST        3  33X   
7          OP  2011-01-01T00:04:25.000         AV&FOOTHILL BLVD        2  24X   
8          OP  2011-01-01T00:05:13.000         AV&FOOTHILL BLVD        2  24X   
9          OP  2011-01-01T00:05:51.000              AV&TEVIS ST        3  26Y   
10         OP  2011-01-01T00:06:21.000                 LYNDE ST        2  21Y   
11         OP  2011-01-01T00

#### 3. 根据数据对象的相似性来填充数据值

   这里我们使用的是KNN算法，实体之间的距离计算使用曼哈顿距离。K的值设置为6。我们选取与存在空缺值的向量距离最近的6个向量在对应属性上的平均值来对空缺值进行填充。